# Práctica 11: Agrupamiento de datos
## Gabriela Sánchez Y.

El objetivo del agrupamiento es descubrir sí o no los datos forman grupos con elementos similares de tal forma que todos los grupos difieren entre sí en algunos aspectos. Para poder llevar a cabo un agrupamiento, primero se necesita establecer una o más medidas de similitud entre los datos o alternativamente una o más medidas de distancia. 

El objetivo de esta práctica es seleccionar por lo menos dos algoritmos distintos para llevar a cabo agrupamiento con por lo menos dos atributos del caso de estudio, reportando los hallazgos con gráficas y con medidas de calidad.


In [ ]:



import ssl
import pandas as pd
from sklearn import metrics
from numpy.random import seed
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from numpy import isnan, nan, take, where


seed(42)
if getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context
d = pd.read_csv("https://elisa.dyndns-web.com/teaching/comp/datasci/modelar.csv")
keep = ['CF1op', 'hrsNum'] # dos atributos numéricos para comenzar
d = d.loc[:, keep]
d = d.dropna() # quitar los NAs
x = d.values
k = 3 # pedimos tres grupos
m = KMeans(init = 'random', n_clusters = k, n_init = 10)
m.fit(x)
centroides = m.cluster_centers_
grupos = m.predict(x)
plt.figure(1)
plt.scatter(centroides[:, 0], centroides[:, 1], marker='x', s=150, linewidths=3, color='black', zorder=10)
colores = ['r', 'g', 'b']
for g in range(k):
    incl = where(grupos == g)[0]
    print(len(incl), "integrantes en grupo", g)
    grupo = take(x, incl, 0)
    plt.scatter(grupo[:, 0], grupo[:, 1], marker='o', s=20, linewidths=2, color=colores[g], alpha = 0.3, zorder=5)
plt.show()
print(metrics.silhouette_score(x, grupos, metric='euclidean'))

